In [ ]:
import random as rd

inventaire = {"(1) Potions de soin":3, "(2) Potion de régénération":1, "(3) Potion de défense":2, "(4) Potion de rage":2, "(5) Piment":1}
invPotiondeSoin = {"(1) Potion de soin Faible":2, "(2) Potion de soin Moyen":1, "(3) Potion de soin Elevé":0}

def YesorNo():
    while True:
        x = input("(Y / N) ? ")
        if x == "Y":
            return True
        elif x == "N":
            return False
        else:
            print("Veuillez entrer une réponse valide.")
            continue

def Critique(critluck, dmg):
    crit = False
    critdmg = rd.random() + 1
    if rd.random() < critluck:
        crit = True
        DamageDealt = int(dmg * critdmg)
    else:
        DamageDealt = int(dmg)
    return DamageDealt, crit

class Personnage:
    def __init__(self, nom, HP, DEF, DMG, critluck, luck, EXP = 0, lvl = 1):
        self.nom = nom
        self.HP = HP
        self.maxHP = HP
        self.DEF = DEF
        self.maxDEF = DEF
        self.DMG = DMG
        self.maxDMG = DMG
        self.critluck = critluck
        self.luck = luck
        self.EXP = EXP
        self.lvl = lvl
        self.Bloquer = False
        self.Parer = False
        self.Esquive = False
        self.ArmeLancée = False
        self.Fuite = False
        self.FireDOT = int(self.DMG*0.35)
        self.AcidDOT = int(self.DMG*0.70)
        self.PoisonDOT = int(self.DMG*0.25)
        
    def estVivant(self):
        return self.HP > 0

    def LVLUP(self):
        print("Vous avez augmenté de niveau ! ")
        self.lvl += 1
        self.EXP = 0
        self.maxHP = int(self.maxHP * 1.10)
        self.HP = self.maxHP
        self.maxDEF = int(self.maxDEF * 1.20)
        self.maxDMG = int(self.maxDMG * 1.25)

    def PassifBlockATK(self):
        if rd.random() < 0.20:
            self.Bloquer = True
        else:
            self.Bloquer = False

#-------------------- Actions --------------------

    def AttaqueBasique(self, target): #Action 1
        if self.nom == "Magnus" or self.nom == "Salie":
            print("Attaque Basique")
        DamageDealt, crit = Critique(self.critluck, self.DMG)
        return target.DégâtsSubis(DamageDealt, crit)


    def LancerProjectile(self, target): #Salie Action 2

        while True:
            print("Choisissez un projectile : ")
            print("1 : Tonneau explosif")
            print("2 : Boule de feu")
            print("3 : Potion jetable d'acide")
            print("4 : Parfum empoisonné")
            choixProjectile = input("Votre projectile : ")

            if choixProjectile == "1": #Tonneau explosif
                print("Tonneau explosif")
                DamageDealt, crit = Critique(self.critluck, self.DMG)
                DamageDealt = int(DamageDealt*0.9)
                return target.DégâtsSubis(DamageDealt, crit)


            if choixProjectile == "2": #Boule de feu
                target.FireCondition = True
                target.FireTours = 3
                print("Boule de feu")
                DamageDealt, crit = Critique(self.critluck, self.DMG)
                DamageDealt = int(DamageDealt*0.75)
                print(target.DégâtsSubis(DamageDealt, crit))
                return f"{target.nom} prendra {self.FireDOT} dégâts de feu pendant 3 tours. "

                
            if choixProjectile == "3": #Potion jetable d'acide
                print("Potion jetable d'acide")
                target.AcidCondition = True
                target.AcidTours = 3
                DamageDealt, crit = Critique(self.critluck, self.DMG)
                DamageDealt = int(DamageDealt*0.30)
                print(target.DégâtsSubis(DamageDealt, crit))
                return f"{target.nom} prendra {self.AcidDOT} dégâts d'acide pendant 3 tours. "

                
            if choixProjectile == "4": #Parfum de poison
                target.PoisonCondition = True
                print("Parfum empoisonné")
                return f"{target.nom} prendra {self.PoisonDOT} dégâts de poison à chaque tour."
            
            else:
                print("Veuillez entrer un nombre valide.")
                continue


    def ParerUneAttaque(self): #Magnus Action 2
        self.ParerTours -= 1
        if self.ParerTours == 0:
            self.ParerCondition = False
            print("(Dernière chance de parer)")
        if rd.random() < self.luck: #0.6
            return True
        else:
            return False


    def LancerArme(self, target): #Magnus Action 3
        self.ArmeLancée = True
        DamageDealt, crit = Critique(self.critluck, self.DMG)
        DamageDealt = int(DamageDealt*2)
        self.DMG = int(self.DMG/2)
        return target.DégâtsSubis(DamageDealt, crit)


    def UtiliserConsommable(self): #Action 3 / 4
        global inventaire
        global invPotiondeSoin
        while True:
            print("Quel consommable utiliser ? ")
            for key,value in inventaire.items():
                print(f"{key} : {value}")
            choixConsommable = input("Votre choix : ")
            print()

            if choixConsommable == "1": #Soin
                if inventaire["(1) Potions de soin"] == 0: #Soin
                    print("Vous n'avez plus de potion de soin ! ")
                    continue
                while True:
                    print("Quelle taille de potion de soin ? ")
                    for key,value in invPotiondeSoin.items():
                        print(f"{key} : {value}")
                    choixPotionDeSoin = input("Choisissez une taille de potion : ")

                    if choixPotionDeSoin == "1":
                        if invPotiondeSoin["(1) Potion de soin Faible"] != 0:
                            self.HP = int(self.HP + (self.HP*0.20))
                            if self.HP > self.maxHP:
                                self.HP = self.maxHP
                            invPotiondeSoin["(1) Potion de soin Faible"] -= 1
                            inventaire["(1) Potions de soin"] -= 1
                            return f"Vous vous soignez de {int(self.maxHP*0.20)}PV. Votre vie : {self.HP}"
                        else:
                            print("Vous n'avez plus de potion de soin faible ! ")
                            print()
                            continue

                    if choixPotionDeSoin == "2":
                        if invPotiondeSoin["(2) Potion de soin Moyen"] != 0:
                            self.HP = int(self.HP + (self.maxHP*0.35))
                            if self.HP > self.maxHP:
                                self.HP = self.maxHP
                            invPotiondeSoin["(2) Potion de soin Moyen"] -= 1
                            inventaire["(1) Potions de soin"] -= 1
                            return f"Vous vous soignez de {int(self.maxHP*0.35)}PV. Votre vie : {self.HP}"
                        else:
                            print("Vous n'avez plus de potion de soin moyen ! ")
                            print()
                            continue

                    if choixPotionDeSoin == "3":
                        if invPotiondeSoin["(3) Potion de soin Elevé"] != 0:
                            self.HP = int(self.HP + (self.maxHP*0.50))
                            if self.HP > self.maxHP:
                                self.HP = self.maxHP
                            invPotiondeSoin["(3) Potion de soin Elevé"] -= 1
                            inventaire["(1) Potions de soin"] -= 1
                            return f"Vous vous soignez de {int(self.maxHP*0.50)}PV. Votre vie : {self.HP}"
                        else:
                            print("Vous n'avez plus de potion de soin élevé ! ")
                            print()
                            continue

                    else:
                        print("Veuillez entrer un nombre valide.")
                        continue

            elif choixConsommable == "2": #Regen
                if inventaire["(2) Potion de régénération"] != 0:
                    self.RegenCondition = True
                    self.RegenTours = 5
                    inventaire["(2) Potion de régénération"] -= 1
                    return f"Vous vous soignerez de {int(self.maxHP*0.15)} pendant 5 tours. Votre vie : {self.HP}."
                else:
                    print("Vous n'avez plus de potion de régénération ! ")
                    continue

            elif choixConsommable == "3": #DEFUp
                if inventaire["(3) Potion de défense"] != 0:
                    self.DEFCondition = True
                    self.DEFTours = 5                
                    self.DEF = int(self.maxDEF*1.35)
                    inventaire["(3) Potion de défense"] -= 1
                    return f"Vous augmentez votre défense de 35% pendant 5 tours. Votre défense : {self.DEF}"
                else:
                    print("Vous n'avez plus de potion de défense ! ")
                    continue

            elif choixConsommable == "4": #DMGUp
                if inventaire["(4) Potion de rage"] != 0:
                    self.RageCondition = True
                    self.RageTours = 5
                    if self.ArmeLancée == True:
                        self.DMG = int((self.maxDMG/2)*1.35)
                    else:
                        self.DMG = int(self.maxDMG*1.35)
                    inventaire["(4) Potion de rage"] -= 1
                    return f"Vous êtes enragé(e) ! Vos attaques infligeront 35% de dégats supplémentaires pendant 5 tours. Vos dégâts : {self.DMG}."
                else:
                    print("Vous n'avez plus de potion de rage ! ")
                    continue
            
            elif choixConsommable == "5": #Esquive
                if inventaire["(5) Piment"] != 0:
                    self.EsquiveCondition = True
                    self.EsquiveTours = 5
                    inventaire["(5) Piment"] -= 1
                    return f"Vous avez {int(self.luck*100)}% de chance d'esquiver l'attaque ennemie pendant 5 tours. "
                else:
                    print("Vous n'avez plus de piment ! ")
                    continue

            else:
                print("Veuillez entrer un nombre valide.")
                print()
                continue


    def TentativeFuite(self):
        ChanceFuite = self.luck/2
        if rd.random() < ChanceFuite: #Salie : 0.75/2, Magnus : 0.6/2
            self.Fuite = True
            return "Vous avez réussi à vous enfuir ! "
        else:
            return "Vous ne parvenez pas à vous enfuir. "

#-------------------- Buffs --------------------

    def Régénération(self):
        self.HP += int(self.maxHP*0.15)
        if self.HP > self.maxHP:
            self.HP = self.maxHP
        print(f"Vous vous soignez de {int(self.maxHP*0.15)} points de vie. ")
        self.RegenTours -= 1
        if self.RegenTours != 0:
            return f"Il vous reste {self.RegenTours} tours de régénération. "
        else:
            self.RegenCondition = False
            return "La régénération prend fin. "

    def AugmentationDEF(self):
        self.DEFTours -= 1
        if self.DEFTours != 0:
            return f"Il vous reste {self.DEFTours} tours de défense augmentée. "
        else:
            self.DEFCondition = False
            self.DEF = self.maxDEF
            return "Votre défense revient à la normale. "
    
    def AugmentationDMG(self):
        self.RageTours -= 1
        if self.RageTours != 0:
            return f"Il vous reste {self.RageTours} tours de rage. "
        else:
            self.RageCondition = False
            if self.ArmeLancée:
                self.DMG = int(self.maxDMG / 2)
            else:
                self.DMG = self.maxDMG
            return "Vos dégâts reviennent à la normale. "

    def Esquiver(self):
        self.EsquiveTours -= 1
        if self.EsquiveTours == 0:
            self.EsquiveCondition = False
            print("(Dernière chance d'esquiver)")
        if rd.random() < self.luck: #Salie : 0.75, Magnus : 0.6
            return True
        else:
            return False

#-------------------- Debuffs --------------------

    def Enflammé(target, self):
        target.HP = int(target.HP - self.FireDOT)
        target.FireTours -= 1
        if target.FireTours == 0:
            target.FireCondition = False
        return ""

    def Acidifié(target, self):
        target.HP = int(target.HP - self.AcidDOT)
        target.AcidTours -= 1
        if target.AcidTours == 0:
            target.AcidCondition = False
        return ""
            
    def Empoisonné(target, self):
        target.HP = int(target.HP - self.PoisonDOT)

#-------------------- Prise de Dégâts --------------------
    
    def DégâtsSubis(self, DamageDealt, crit):
        if self.nom != 'Salie' and self.nom != 'Magnus':
            if self.PassifBlockATK():
                return "Attaque bloquée ! "
            
        if hasattr(self, "ParerCondition") and self.ParerCondition:
            if self.ParerUneAttaque():
                return "Attaque parée ! "

        if hasattr(self, "EsquiveCondition") and self.EsquiveCondition:
            if self.Esquiver():
                return "Attaque esquivée ! "

        if crit:
            self.HP -= DamageDealt
            return f"{self.nom} prend {DamageDealt} dégâts (coup critique ! ). Sa vie restante : {self.HP}"
        else:
            self.HP -= max(0, DamageDealt - self.DEF)
            return f"{self.nom} prend {max(0, DamageDealt - self.DEF)} dégâts. Sa vie restante : {self.HP}"
        
#-------------------- Inventaire -> get consommables --------------------

def ObtenirConsommable(amount):
    global inventaire
    global invPotiondeSoin
    print("Vous obtenez les consommables suivants : ")
    for i in range(amount):
        ConsommableObtenu = rd.choices(list(inventaire.keys()), weights = [0.20, 0.15, 0.30, 0.25, 0.10])[0]
        inventaire[ConsommableObtenu] += 1
        if not ConsommableObtenu == "(1) Potions de soin":
            print(f"{ConsommableObtenu[4:]}")
        else:
            SoinObtenu = rd.choices(list(invPotiondeSoin.keys()), weights = [0.60, 0.25, 0.15])[0]
            invPotiondeSoin[SoinObtenu] += 1
            print(f"{SoinObtenu[4:]}")
    print()

    print("Votre inventaire : ")
    print()
    for key, value in inventaire.items():
        print(f"{key[4:]} : {value}")
    print()
    if "(1) Potions de soin" in ConsommableObtenu:
        print("Vos potions de soin : ")
        for key, value in invPotiondeSoin.items():
            print(f"{key[4:]} : {value}")
    return ""

#-------------------- Invoc de Monstre --------------------

def SummonMonstre(zone, nom = None):
    HP_range = [40 + 20 * (zone - 1), 60 + 20 * (zone - 1)]
    DEF_range = [1 + zone, 3 + zone]
    DMG_range = [10 + 4 * (zone - 1), 14 + 4 * (zone - 1)]
    if nom is not None:
        if nom == "Mimic": #stats du Mimic (stats zone + 1)
            nom = "Mimic"
            HP = rd.randint(40 + 20 * (zone), 60 + 20 * (zone))
            DEF = rd.randint(1 + zone+1, 3 + zone+1)
            DMG = rd.randint(10 + 4 * (zone), 14 + 4 * (zone))
        else: #monstre avec nom donné
            nom = nom
            HP = rd.randint(*HP_range)
            DEF = rd.randint(*DEF_range)
            DMG = rd.randint(*DMG_range)
    else: #monstre nom aleatoire
        nom = rd.choice(monstre_zone[zone-1])
        HP = rd.randint(*HP_range)
        DEF = rd.randint(*DEF_range)
        DMG = rd.randint(*DMG_range)
    critluck = 0.20 + (0.05 * (zone-1))
    global Monstre
    Monstre = Personnage(nom, HP, DEF, DMG, critluck, 0.40, 0, zone)

#-------------------- main --------------------


def isMimic():
    if rd.random() <= 0.25:
        return True
    else:
        return False
    
def Trésor(zone):
    print("Vous obtenez un trésor ! ")
    print("Voulez-vous l'ouvrir ? ")
    if YesorNo():
        if isMimic():
            print("C'était un Mimic, vous vous êtes fait avoir ! ")
            print()
            mainCombat(zone, "Mimic")
        else:
            return ObtenirConsommable(3)
    else:
        return "Vous laissez le coffre. "
    
def ActionJoueur():    
    while True:

        if ChoixHéros == "1":
            print("Quelle action ? Entrez un nombre : ")
            print("(1) Attaque basique")
            print("(2) Lancer un objet")
            print("(3) Utiliser un consommable")
            print("(4) Tentative de fuite")
        if ChoixHéros == "2":
            print("Quelle action ? Entrez un nombre : ")
            print("(1) Attaque basique")
            print("(2) Chance de parer (3 fois)")
            print("(3) Lancer votre arme (2x dégâts, mais vous infligerez ensuite ATK/2)")
            print("(4) Utiliser un consommable")
            print("(5) Tentative de fuite")
        
        choix = input("Votre action : ")
        print()

        if choix == "1": #Attaque basique
            print(Héros.AttaqueBasique(Monstre))
            break
        
        elif choix == "2" and ChoixHéros == "1": #Lancer un projectile (Salie)
            print(Héros.LancerProjectile(Monstre))
            break
        
        elif choix == "2" and ChoixHéros == "2": #Parer (Magnus)
            Héros.ParerCondition = True
            Héros.ParerTours = 3
            print(f"Vous avez {int(Héros.luck*100)}% de chance de parer les trois prochaines attaques. ")
            break
        
        elif choix == "3" and ChoixHéros == "2": #Lancer l'arme (Magnus)
            if Héros.ArmeLancée:
                print("Vous avez déjà lancé votre arme ! ")
                continue
            else:
                print("Vous avez lancé(e) votre arme ! ")
                print(Héros.LancerArme(Monstre))
                break
            
        elif (choix == "3" and ChoixHéros == "1") or (choix == "4" and ChoixHéros == "2"): #Utiliser un consommable
            if all(value == 0 for value in inventaire.values()):
                print("Vous n'avez plus de consommable ! ")
                continue
            print(Héros.UtiliserConsommable())
            break
        
        elif (choix == "4" and ChoixHéros == "1") or (choix == "5" and ChoixHéros == "2"): #Fuire
            print(Héros.TentativeFuite())
            break
        
        else:
            print("Veuillez entrer un nombre valide.")
            continue

def mainCombat(zone, Monstre_nom = None):
    SummonMonstre(zone, Monstre_nom)
    print(f"Vous affrontez {Monstre.nom} ! ")
    print(f"Ses statistiques : PV : {Monstre.HP}, DEF : {Monstre.DEF}, ATK : {Monstre.DMG}, Chance de coup critique : {Monstre.critluck}, Chance : {Monstre.luck}")
    print()
    global Héros
    Tour = 0

    while Héros.estVivant() and Monstre.estVivant():
        Tour += 1
        print("--------------------")
        print(f"Tour : {Tour}")
        print("--------------------")
        print()

        if hasattr(Héros, "RegenCondition") and Héros.RegenCondition:
            print(Héros.Régénération())
        if hasattr(Héros, "DEFCondition") and Héros.DEFCondition:
            print(Héros.AugmentationDEF())
        if hasattr(Héros, "RageCondition") and Héros.RageCondition:
            print(Héros.AugmentationDMG())

        print()
        print(f"{Héros.nom} -> PV : {Héros.HP}/{Héros.maxHP}, DEF : {Héros.DEF}, ATK : {Héros.DMG}")
        print()

        ActionJoueur()
        if Héros.Fuite:
            break
        print()

        if not Monstre.estVivant():
            print(f"{Monstre.nom} est mort, Vous avez gagné ! ")
            Héros.EXP += 25
            print(f"Vous obtenez 25 points d'expérience ({Héros.EXP}/100). ")
            if Héros.EXP == 100:
                print(Héros.LVLUP())
            if Héros.ArmeLancée:
                print("Vous ramassez votre arme. ")
                Héros.ArmeLancée = False
            print(Trésor(zone))
            return ""

        if hasattr(Monstre, "FireCondition") and Monstre.FireCondition:
            Monstre.Enflammé(Héros)
            print(f"{Monstre.nom} prend {Héros.FireDOT} dégâts de feu. ")
        if hasattr(Monstre, "AcidCondition") and Monstre.AcidCondition:
            Monstre.Acidifié(Héros)
            print(f"{Monstre.nom} prend {Héros.AcidDOT} dégâts d'acide. ")
        if hasattr(Monstre, "PoisonCondition") and Monstre.PoisonCondition:
            Monstre.Empoisonné(Héros)
            print(f"{Monstre.nom} prend {Héros.PoisonDOT} dégâts de poison. ")

        print()
        print(f"{Monstre.nom} -> PV : {Monstre.HP}/{Monstre.maxHP}, DEF : {Monstre.DEF}, ATK : {Monstre.DMG}")
        print()

        if not Monstre.estVivant():
            print(f"{Monstre.nom} est mort, Vous avez gagné ! ")
            Héros.EXP += 25
            print(f"Vous obtenez 25 points d'expérience ({Héros.EXP}/100). ")
            if Héros.EXP == 100:
                print(Héros.LVLUP())
            if Héros.ArmeLancée:
                print("*Vous ramassez votre arme. ")
                Héros.ArmeLancée = False
            print(Trésor(zone))
            return ""
        
        print("L'ennemi attaque ! ")
        print(Monstre.AttaqueBasique(Héros))
        print()

        if not Héros.estVivant():
            print("Vous êtes mort ! ")
            return False
        
                

while True: #Choix du personnage
    print("Quel personnage voulez-vous choisir ? ")
    print("1 : Salie (PV : 100, DEF : 4, ATK : 10, ChanceCritique : 25, Chance : 75)")
    print("2 : Magnus (PV : 120, DEF : 6, ATK : 8, ChanceCritique : 30, Chance : 60)")
    ChoixHéros = input("Entrez un nombre : ")
    print()

    if ChoixHéros == "1":
        Héros = Personnage("Salie", 100, 4, 10, 0.25, 0.75) #nom, PV, DEF, ATK, %crit, %luck
        print("Vous avez choisi Salie.")
        break
    elif ChoixHéros == "2":
        Héros = Personnage("Magnus", 120, 6, 8, 0.30, 0.60) #nom, PV, DEF, ATK, %crit, %luck
        print("Vous avez choisi Magnus.")
        break
    else:
        print("Veuillez entrer un nombre valide. ")
        print()
        continue
print()



mainCombat(1)

In [ ]:

print("Bienvenue dans: _The lost crown_")

def trame_introduction():
    Introduction = [
    "L'histoire prend place au 12e siècle après J.C.. ", 
    "Une rumeur court dans le village de Ravensbrook en Norvège : ", 
    "un roi puissant nommé Harald aurait régné sur la contrée il y 200 ans. ", 
    "Un joyau d'une valeur inestimable aurait été découvert dans la montagne du nord : ", 
    "une montagne où résident de nombreuses créatures terrifiantes. ", 
    "Ce roi aurait donc trouvé ce joyau en réalité maudit et l'aurait placé sur sa couronne. ", 
    "Seulement, les monstres de la montagne sont puissants. ", 
    "Son royaume fût assiégé par ces créatures, le poussant à fuire. ", 
    "Le roi serait parti à la hâte, délaissant le joyau dans son château au sommet de la Montagne Enchantée. "
    ] #9 objets
    for phrase in Introduction: #elements de la liste 
        print(phrase)
        time.sleep(2)#attente de 2 secondes avant affichage de la prochaine ligne
def commencement_histoire():

    trame_intro_2 = [ 
    "*Le héros se réveille un matin. *",
    " ",
    "SALIE :",
    "part en quête d'ingrédients préparer une potion revitalisante pour un client lorqu'elle tente d'attraper un guérit-tout au pied d'un arbre.",
    "Un rayon de soleil surgit et attire son attention. Elle se retourne et voit devant elle au loin la Montagne Enchantée. Elle repense au joyau:",
    ]

    répliques_Salie =[
    "Ce joyau doit bien exister. Là quelque part la haut. ",
    "Ce ne sont pas quelques grizzlis ou quelques reptiles de 3 mètres de long qui vont me manger...",
    "J'ai déjà combattu pour notre village. Et d'ailleurs j'en suis la meilleure. ",
    " ",
    "Je suis décidée, je pars demain. ",
    "Ainsi Salie rassemble ses affaires et son équipement. Le soir même, elle se rendit chez Garry, le vieux sage du village.",
    "Il est l'un des descendants du fondateur du village nommé Raymond ERIKSON.",
    "Un homme fort et intelligent, aux capacités de survie toutes aussi impressionnantes que ses capacités de combat. ",
    "Garry:",
    "HORS DE QUESTION !",
    "Salie:",
    "S'il te plaît! s'il te plaît! s'il te plaît! s'il te plaît!",
    "Garry:",
    "Tu me rappelles moi dans ma jeunesse...",
    "A me précipiter à l'aventure sans même réfléchir ou penser aux conséquences",
    "Cette cicatrice sur mon visage... a été faite par une créature extrêmement dangereuse",
    "un monstre...",
    " ",
    "Je n'ai pas de descendants, prend mon héritage, l'épée des Erikson !", 
    "Salie:",
    "Je te revaudrai ça grand-pèr..",
    "Garry:",
    "Tu crois vraiment que je vais te la confier de cette manière ?",
    "L'épée choisit son maître et est capable de libérer un pouvoir dévastateur ! ", 
    "Si tu la veux, tu vas devoir la mériter !",
    ]

    répliques_Magnus = [
    "MAGNUS:"
    "part couper du bois dans la forêt avoisinnante afin de remplir son stock de buches pour sa cheminée.",
    "Après s'être aventuré au coeur de celle-ci, un rugissement lointaint attire son attention.",
    "Il lève les yeux et voit devant lui au loin, la _montagne enchantée_ ",
    "Il repense au joyau:",
    "Ce caillou doit bien exister. Là! quelque part la haut!",
    "Je suis certain que la moitié des bêtes de la montagne ne peuvent pas rivaliser pas contre moi !",
    "au loin: ",
    "Rooooaaaaaaarrr !...",
    " ",
    "Magnus:",
    "AAH ! C'était quoi ça !? ",
    "Ce n'était pas un ours. Mais alors, qu'est ce que ça peut bien être ? ",
    "Si je veux en savoir plus, je sais ce qu'il me reste à faire...",
    "J'ai déjà combattu pour notre village. Mes capacités aux combats m'ont permis de survivre jusqu'ici.",
    " ",
    "Ainsi Magnus rassemble ses affaires et son équipement. Le soir même, il se rendit chez Garry, le vieux sage du village.",
    "Il est l'un des descendants du fondateur du village nommé Raymond ERIKSON.",
    "Un homme fort et intelligent, aux capacités de survie toutes aussi impressionnantes que ses capacités de combat.",
    "*5 min plus tard",
    "Garry:",
    "HORS DE QUESTION !",
    "Magnus:",
    "S'il te plaît! s'il te plaît! s'il te plaît! s'il te plaît!",
    "Garry:",
    "Tu me rappelles moi dans ma jeunesse...",
    "A me précipiter à l'aventure sans même réfléchir ou penser aux conséquences",
    "Cette cicatrice sur mon visage... a été faite par une créature extrêmement dangereuse",
    "un monstre...",
    " ",
    "Je n'ai pas de descendants, prend mon héritage, l'épée des Erikson !", 
    "Magnus:",
    "merci pap..",
    "Garry:",
    "Tu crois vraiment que je vais te la confier de cette manière ?",
    "L'épée choisit son maître et est capable de libérer un pouvoir dévastateur ! ", 
    "Si tu la veux, tu vas devoir la mériter !",
    ]

    if ChoixHéros=="1":#Affichage du texte pour Salie
        for phrase in répliques_Salie:
            print(phrase)
            time.sleep(1.5)
            input("Tapez ENTRER")

    elif ChoixHéros=="2":#Affichage du texte pour Magnus
        for phrase in répliques_Magnus:
            print(phrase)
            time.sleep(1.5)
            input("Tapez ENTRER")
def transition_1er_combat():
    trame = [
    "*6 mois se sont écoulés",
    "Garry:",
    "Te voilà novice dans le rang des chasseurs. Chaque cible que tu éliminera te fera monter en rang. Ton savoir n'en sera qu'amélioré...",
    "Sache tout de même que ton premier but est de protéger le village.",
    "Comme par ton entraînement tu a su le comprendre, un grand pouvoir implique de grandes responsabilités...",
    "*Soudainement, un rugissement provenant du coeur du village attire votre attention.",
    "Oh non...",
    "Garry:", 
    "C'est l'occasion idéale, va sauver notre village et rapporte moi ce joyau!!",
    " ",
    ]
    for phrase in trame:
            print(phrase)
            time.sleep(2.5)   
#premier_combat: vs Gobelin              
def fin_introduction():
    répliques = [
    "Salie:",
    "OUAHH ! Je viens vraiment de tuer ce monstre !!?",
    "Oh ! Qu'est ce qui scintille juste là ?", 
    "*Lorsque vous neutralisez une créature, vous pouvez choisir d'ouvrir ou non le coffre qui vous est offert en récompense",
    "Cool, je pourrais en avoir besoin un de ces quatre",
    "Garry:",
    " "
    "Pas mal pour une première. Mais On dirait qu'on a eu chaud. On est pas en sécurité ici.",
    "L'avenir de notre village est incertain, il repose entre tes mains désormais. Va! Et ne reviens pas sans avoir détruit ce joyau !",
    "Salie:",
    "Et dire que je voulais juste une retraite paisible...",
    "Garry:",
    "Mais après tout, ce joyau n'apporte rien de bon. Les créatures s'excitent et se rapprochent du village.",
    "Salie:",
    "C'est compris Garry ! Dis à tout le monde qu'ils n'ont rien à craidre car je suis là ! "       
    " ",
    "*Ainsi le héros commence sa quête.",
    " ",
    ]

    if ChoixHéros=="1":#Affichage du texte pour Salie
        for phrase in répliques:
            print(phrase)
            time.sleep(2)
            input("Tapez ENTRER")

    elif ChoixHéros=="2":#Affichage du texte pour Magnus
        répliques[0] = "Magnus:"
        répliques[1] = "OUAHH OK ! C'est bien plus impressionnant que ce que je pensais...°~°"
        répliques[9] = "Magnus:"
        répliques[13] = "Magnus:"
        répliques[14] = "J'ai ressentis les fruits de mon entrainement aujoud'hui... Ca va mal finir pour ces fichus monstres !"
        for phrase in répliques:
            print(phrase)
            time.sleep(2)
            input("Tapez ENTRER")


def prairie_niveau_1():
 
    trame_prairie_1 = [
    "*PRAIRIE: Niveau 1",
    "Salie:", 
    "OK...",
    "Va falloir que je surveille mes arrières. Je me suis jamais autant approchée de la montagne.",
    "*Poursuit son entrainement.",
    "*se repose 2h plus tard.",
    "*Puis part ensuite en quête de nourriture pour les prochains jours.",
    "Salie:",
    "Je devrais cherher un point d'eau",
    "*Trouve un ruisseau en pénétrant à travers les bois qui succèdent à la prairie.",
    "Salie:",
    "*Sors sa gourde",
    "*Une bête surgit de derrière les buissons !",
    "Salie:",
    "*Se met en garde.",
    "Oh ce n'est qu'une biche... Qu'elle est mignonne.",
    "SLAAK ! Un monstre vient de surgir des buissons et d'embraucher violemment la biche avec ses griffes aserrées !!",
    "...! *Regarde le spectacle avec émotion",
    "*Cris de la biche",
    "*Gloussements du monstre qui déchicte sa proie",
    "Il faut que j'y aille. Il a l'air d'avoir l'avantage ici...",
    "Tu vas me le payer sale pourriture !",
    "Salie:",   
    "Rentre au camp et repense à ce qui s'est passé.",
    "*Deux jours se sont écoulés.",
    "Bon, je sais par où je dois passer pour atteindre le coeur de la forêt",
    "*Elle se munit de ses affaires et se met en route", 
    "*Devant au loin, un gros rocher noir.",
    "Mais... OH ! Le monstre, il s'est endormi en plein milieu de la route !",
    "*En tentant une approche furtive:",
    "Monstre: ROAAAAAAR !!!",
    " ",
    "*C'est un piège !",
    "Il faut que je l'attire vers la plaine! J'aurai l'avantge",
    "*Vous arrivez de justesse à la plaine. Mais le monstre ne se laisse pas devancer si facilement",
    " ",
    " ",   
    ]

    if ChoixHéros=="1":#Affichage du texte pour Salie
        for phrase in trame_prairie_1:
            print(phrase)
            time.sleep(1.5)

    elif ChoixHéros=="2":#Affichage du texte pour Magnus
        trame_prairie_1[1] = "Magnus:"
        trame_prairie_1[7] = "Magnus:"
        trame_prairie_1[10] = "Magnus:"
        trame_prairie_1[13] = "Magnus:"
        trame_prairie_1[15] = "Oh une biche! Ca pourrait être mon repas de ce soir"
        trame_prairie_1[22] = "Magnus:"
        trame_prairie_1[26] = "*Il se munit de ses affaires et se met en route"
        for phrase in trame_prairie_1:
            print(phrase)
            time.sleep(1.5)
#Combat contre le Stritominus
def prairie_niveau_2():
    
    répliques_prairie_fin_de_zone = [
    "Salie:",
    "J'espère seulement qu'il y a pas la maman dans les parages...hihihi ^-^",
    "*La terre se met à trembler",
    "Salie:",
    "OK BON AUTANT EN FINIR TOUT DE SUITE...¨_¨ ",
    "*Un Stritominus 2 fois plus rapide arrive à toute vitesse !",
    " ",
    ]

    if ChoixHéros=="1":#Salie
        for phrase in répliques_prairie_fin_de_zone:
            print(phrase)
            time.sleep(0.5)
            input("Tapez ENTRER")

    elif ChoixHéros=="2":#Magnus
        répliques_prairie_fin_de_zone[0] = "Magnus:"
        répliques_prairie_fin_de_zone[3] = "Magnus:"
        for phrase in répliques_prairie_fin_de_zone:
            print(phrase)
            time.sleep(0.5)
            input("Tapez ENTRER")
#Combat final prairie 


def foret_partie_1():
    
    print("*ZONE 2: Au coeur de la forêt")
    trame_forêt_1 = [
    "Salie:",
    "Je devrais me trouver un coin sûr. Vu sur quoi je viens de tomber.",
    "*En s'enfonçant dans la forêt, Salie trouve une grotte au loin, cachée au pied d'un énorme arbre.",
    "Super ! J'espère seulement qu'il y aura pas de locataire(s)",
    "*La grotte est étroite, mais aucune trace du passage de qui ou quoi que ce soit.",
    "Salie:", 
    "Je vais me poser un instant. Les combats d'hier m'ont fatigué.",
    " ",
    " ",
    " ",
    "*6h plus tard",
    "Salie (*s'étire):",
    "Huh.. J'ai dormi combien detemps ?",
    "Mince ! le soleil est presque couché !",
    "Je devrais pas m'aventurer dehors, je connais pas assez bien les environs.",
    "Faut que je barricade l'entrée de la grotte.",
    "1 heure plus tard, avec quelques branches et de la mousse, vous avez réussi à camoufler l'entrée de la grotte.",
    "Salie:",
    "Bon cette barricade empêcherait même pas un écureuil d'entrer... Mais l'avantage c'est que je suis fondu dans le décor",
    "*Les prédateurs se réveillent, les loups hurlent à la pleine lune. Le vent souffle.",
    "Salie:",
    "Je vais continuer sur ma lancée et me reposer jusqu'à l'aube.",
    ]

    if ChoixHéros=="1":#Salie
        for phrase in trame_forêt_1:
            print(phrase)
            time.sleep(2)

    elif ChoixHéros=="2":#Magnus
        trame_forêt_1[0] = "Magnus:"      
        trame_forêt_1[2] = "*En s'enfonçant dans la forêt, Magnus trouve une grotte au loin, cachée au pied d'un énorme arbre." 
        trame_forêt_1[5] = "Magnus:"
        trame_forêt_1[6] = "Magnus: Je vais me poser un instant. Je vais repenser aux K.O auquel ces monstres ont gouté hier (*baille) "
        trame_forêt_1[7] = "Oui bon je suis exténué en réalité..."
        trame_forêt_1[11] = "Magnus(*s'étire):"
        trame_forêt_1[17] = "Magnus:"
        trame_forêt_1[20] = "Magnus:"
        for phrase in trame_forêt_1:
            print(phrase)
            time.sleep(2)
def foret_partie_2_combat_fin():
    répliques_foret_p2 = [
    "*2 heures s'écoulent:",
    "Salie:", 
    "Impossible de fermer l'oeil, il fait pas hyper chaud non plus.",
    "'*bruits de pas' ",
    "Salie:",
    "C'était quoi ?",
    "*Les bruits de pas se rapprochent de l'entrée de la grotte",
    "Salie:",
    " °_°...",
    "(*Cette chose doit être massive, on en ressent les secousses lorsqu'elle se déplace.)",
    "Salie:", 
    "*Une poussière lui tombe sur le nez",
    "'ATCHOUM !'",
    "oh oh..",
    "Créature: Mmmmm!!?",
    "*Les bruits de pas proviennent de dessus de la cavité'",
    "La créature est juste au dessus...",
    "La Lune laisse apparaître l'ombre de la créature, que vous voyez à travers un petit interstice.",
    "Une bête affreusement poilue et diforme",
    "*Elle s'en va après un cours instant...",
    "Salie:",
    "Fiou ! J'ai cru que j'all...",
    "Créature: WWRRAAAAAAAA !!",
    "La créature passe son bras à travers la terre et vous attrappe par le col !",
    "Vous vous trouvez nez à nez avec un 'Mordeur' et lui assénez un coup pour le repousser.",
    ]

    if ChoixHéros=="1":#Salie
        for phrase in répliques_forêt_p2:
            print(phrase)
            time.sleep(0.5)

    elif ChoixHéros=="2":#Magnus
        répliques_forêt_p2[1] = "Magnus:"      
        répliques_forêt_p2[4] = "Magnus:"  
        répliques_forêt_p2[7] = "Magnus:"
        répliques_forêt_p2[10] = "Magnus:" 
        répliques_forêt_p2[20] = "Magnus:" 
        for phrase in répliques_forêt_p2:
            print(phrase)
            time.sleep(0.5)
            input()
#Combat contre le Mordeur
def réplique_apres_combat_mordeur():
    répliques_apres_combat_foret = [ # répliques apres combat vs Mordeur
    "J'ai.. ré..u..ssi",
    "_",   
    ]
    for element in répliques_apres_combat_foret: # les elements: "J'ai.. ré..u..ssi" et "*S'évanouit"       #fait défiler
        for character in element:                                                                           #texte caractere par caractere 
            print(character, end = "", flush = True)#pour faire afficher tous caracteres sur la même ligne  #plus lentement
            time.sleep(0.30)
    print("*S'évanouit")


def montagne():
    trame_montagne = [
    "Salie:", 
    "Reprend vaguement ses esprits",
    "Mmmm... où est ce que je suis ?",
    "*Regarde autour d'elle",
    "Euuh ok... La grotte où j'étais jusque là me paraissait un peu plus petite. '=' ",
    "*Regarde vers l'entrée",
    "(°O°) OOOH !!",
    "COMMENT JE SUIS ARRIVEE ICI!!?",
    "*Level up: Niv 4",
    "*Vous voilà dans l'entre d'une grotte, située dans la _Montagne Enchantée_",
    "*Un bruit étrange provient de l'extérieur",
    "Salie:", 
    "On dirait qu'il se passe quelque chose dehors",
    "Aller voir ?",
    ]

    if ChoixHéros=="1":#Affichage du texte pour Salie
        for phrase in trame_montagne:
            print(phrase)
            time.sleep(1.5)

    elif ChoixHéros=="2":#Affichage du texte pour Magnus
        trame_montagne[0] = "Magnus:"                      #modification texte à la ligne 0
        trame_montagne[4] = "*Regarde autour de lui"       #modification texte à la ligne 4 
        trame_montagne[8] = "COMMENT JE SUIS ARRIVE ICI!!?"#modification texte à la ligne 8
        trame_montagne[12] = "Magnus:"                     #modification texte à la ligne 12
        for phrase in trame_montagne:
            print(phrase)
            time.sleep(1.5)
    
    
    print()
    déplacement = 0
    déplacement = input("OUI: 1  /  NON: 2")
    if déplacement == "1":
        trame_déplacement_1 = [
        "Vous avez décidé de vous déplacer jusqu'à la sortie",
        " ",
        "Soudain une créature humanoîde à la gueule énorme portant une massue énorme émerge, chevauchant un loup noir !",
        "Salie:",
        "Ah! Vous avez une sale tête monsieur !",
        "Orc: MOUAAAAAAA !!",
        "Repas réveillé !!",
        "Loup: WWrrraaa !",
        "Salie:",
        "Eh! gentil toutou...tout doux..tout do..",
        ]

        if ChoixHéros=="1":#Affichage du texte pour Salie
            for phrase in trame_déplacement_1:
                print(phrase)
                time.sleep(1.5)

        if ChoixHéros=="2":#Affichage du texte pour Magnus
            trame_déplacement_1[3] = "Magnus:"  #modification texte à la ligne 3
            trame_déplacement_1[8] = "Magnus:"  #modification texte à la ligne 8
            for phrase in trame_déplacement_1:
                print(phrase)
                time.sleep(1.5)

    if déplacement == "2":
        trame_déplacement_2 = [
        "Vous restez au sol",
        "Salie:", 
        "C'est le moment ou jamais..",
        ]
        
        if ChoixHéros=="1":#Affichage du texte pour Salie
            for phrase in trame_déplacement_2:
                print(phrase)
                time.sleep(1.5)

        if ChoixHéros=="2":#Affichage du texte pour Magnus
            trame_déplacement_1[1] = "Magnus:"  #modification texte à la ligne 1
            for phrase in trame_déplacement_2:  
                print(phrase)
                time.sleep(1.5)
#Combat contre l'Orc
def montagne_fin():

    trame_2_montagne = [
    "Salie:",
    "On dirait que cette petite sieste m'a permise de me requinquer !",
    "En quittant la zone de combat, vous levez la tête et apercevez des ruines au sommet de la Monstagne.",
    "Mais c'est le château ! Je suis presque arrivée, encore un petit effort!",
    "En parlant d'effort, j'en fait assez pour aujourd'hui..",
    "Salie: *Regarde le loup allongé à moitié mort.",
    "hehehe...",
    "Loup: *panique",
    "Salie:", 
    "VIENS LA !",
    "Loup: AAAAAAAAA !",
    "*Ainsi Salie chevauche la bête et atteint après quelques heures seulement le château.",       
    ]

    if ChoixHéros=="1":#Affichage du texte pour Salie
        for phrase in trame_2_montagne:
            print(phrase)
            time.sleep(1.75)

    elif ChoixHéros=="2":#Affichage du texte pour Magnus
        trame_2_montagne[0] = "Magnus:"                                                                                 #modification texte à la ligne 0
        trame_2_montagne[3] = "Mais c'est le château que je vois là !? J'y suis presque, encore un petit effort !"      #modification texte à la ligne 3
        trame_2_montagne[5] = "Magnus: *Regarde le loup allongé à moitié mort."                                         #modification texte à la ligne 5
        trame_2_montagne[8] = "Magnus:"                                                                                 #modification texte à la ligne 8
        trame_2_montagne[11] = "*Ainsi Magnus chevauche la bête et atteint après quelques heures seulement le château." #modification texte à la ligne 11
        for phrase in trame_2_montagne:
            print(phrase)
            time.sleep(1.75)

    
def Château_abandonné():
    trame_château = [
    "Salie:",
    "Eh bas voilà vous pouvez bien servir à quelque chose des fo..",
    "*En voyant le château, le monstre déguerpit à toute vitesse",
    "Ok.. Ca n'annonce rien de bon.",
    "Je sais que je suis intimidante mais à ce point là ?",
    "*Les poils de Salie se hérissent",
    "*Une pression hors norme provient de l'intérieur du château",
    "Salie:",
    "Oh ! '°'",
    "Le château est immensément grand",
    "La poussière omniprésente est la marque du temps passé à l'abandon et à la mercie des créatures prédatrices environnantes",
    "Salie *craintive:",
    "Ca y est... nous y sommes.",
    " ",
    " ",
    ]

    if ChoixHéros=="1":#Affichage du texte pour Salie
            for phrase in trame_château:
                print(phrase)   #affiche texte
                time.sleep(1.75)#temps d'attente avant de dérouler la prochaine phrase
  
    elif ChoixHéros=="2":#Affichage du texte pour Magnus
        trame_château[0] = "Magnus:"                                                             #modification texte à la ligne 0
        trame_château[3] = "Bas.. A bas d'accord. Je suis trop imposant pour toi c'est ça !?"    #modification texte à la ligne 3
        trame_château[4] = "C'est ça, cours flipp.."                                             #modification texte à la ligne 4                                         
        trame_château[5] = "*Les poils de Magnus se hérissent"                                   #modification texte à la ligne 5        
        trame_château[7] = "Magnus:"                                                             #modification texte à la ligne 8
        trame_château[11] = "Magnus *craintif:"                                                  #modification texte à la ligne 11
        for phrase in trame_château:
            print(phrase)   
            time.sleep(1.75)

    print("Chargement de la zone finale")
    chargement = [
    ".....",
    ".....",
    ".....",
    ]
    for element in chargement: #, les elements: "...", "...", "...", " ",
        for character in element: 
            print(character, end = "", flush = True)#pour faire afficher tous caracteres sur la même ligne
            time.sleep(0.60)


    trame_entrée_château = [
    "*Cinématique", 
    "Le héros se dirige vers l'entrée du château d'un pas craintif",
    "En arrivant devant la porte d'entrée:",
    "Salie:",  
    "Qu'est ce qui pourrait bien se caher derrière ?",
    "De toute façon il ne me semble pas qu'une bête se plairait dans ce château, endroit reculé..spacieux..et inhabité...",
    "Parfait pour y établir son nid...",
    "Une fois dedans, des dizaines d'ossements humains et animales jonchent le sol.",
    "Salie:",
    "Oh ! Ca empeste à la mort ici.",
    "Il y a un étage composé de plusieurs salles",
    "Peut être y trouverez-vous des indices sur la localisation du joyau...",
    ]

    if ChoixHéros=="1":#Affichage du texte pour Salie
            for phrase in trame_entrée_château:
                print(phrase) 
                time.sleep(2)
  
    if ChoixHéros=="2":#Affichage du texte pour Magnus
        trame_entrée_château[3] = "Magnus:" #modification texte à la ligne 3
        trame_entrée_château[8] = "Magnus" #modification texte à la ligne 3
        for phrase in trame_château:
            print(phrase)   
            time.sleep(1.75)

    trame_fouille = [
    "Vous allez au premier étage.",
    "en entrant dans une bibliothèque",
    "Vous trouvez plusieurs étagères remplies de livres et de parchemins. Au centre de la salle, un pupitre  et une lettre",
    "lettre*: Sur ordre du roi et en tant que son valet, j'ai mis en lieu sûr sa couronne",
    "Dans un coffre caché sous le trone même ! Qui aurait pû y penser ? :)",
    "la clé se trouve là où il n'y pas souvent grand monde, la...",
    "*le texte s'arrête là. Il a dû se passer quelque chose au moment où l'auteur écrivait",
    " ",
    "Vous entrez dans la cuisine",
    "Rien de plus déprimant qu'une cuisine sans rien à manger...",
    "Pas d'indices",
    " ",
    "Vous entrez dans la chambre d'ami.",
    "En vous allongeant sur le lit poussiéreux, quelque chose vous dérange sous le coussin presque entièrement déplumé de l'intérieur:",
    "En le soulevant, vous voyez une clé étrange.",
    "Vous avez obtenu: clé de coffre !",
    ]
    for phrase in trame_fouille:
        print(phrase)
        time.sleep(2)  
    

    trame_combat_dragon = [
    "*Soudain vous êtes surpris par un cri effrayant",
    "*Le château commence à s'écrouler",
    "Salie:",
    "AAAA ! Qu'est ce que c'est !?",
    "Un corps céleste jaillit du sol, explosant la toiture  du château !",
    "Salie:", 
    "Ca y est, c'est maintenant !",
    "BOSS FINAL: Chevalier fantôme",  
    "Salie:",
    "A nous deux sale monstre !!!",
    ] 
    if ChoixHéros=="1":#Affichage du texte pour Salie
        for phrase in trame_combat_dragon:
            print(phrase) 
            time.sleep(2)
  
    if ChoixHéros=="2":#Affichage du texte pour Magnus
        trame_combat_dragon[2] = "Magnus:" #modification texte à la ligne 3
        trame_combat_dragon[5] = "Magnus"  #modification texte à la ligne 5
        trame_combat_dragon[8] = "Magnus"  #modification texte à la ligne 8
        for phrase in trame_combat_dragon:
            print(phrase)   
            time.sleep(2)
#combat final
def trame_finale_apres_boss():
    trame_apres_boss = [
    "L'épée scintille ! ",
    "Son pouvoir s'infiltre à travers les muscles de Salie.",
    "Elle court et bondit sur les décombres, brandit son épée et effectue un saut magistral d'une 40taine de mètres",
    "Le chevalier fantôme prépare son attaque ultime en volant jusqu'au ciel puis fonce en flèche en direction du château", 
    "Slaaash !!! Une lueur éclatante jaillit de cette impact dévastateur !",
    "La lumiere se dissipe. Derrière Salie, le corps du fantôme, inerte, se désintègre",
    "Salie ouvre le coffre sous le trone du roi, prend le joyau maudit et le détruit.²", 
    "*Les montres perdent leur pouvoir et deviennent innoffensifs ",
    "Sauvant ainsi la contrée et ses habitants",
    " ",
    "FIN"
    " ",
    "                       Réalisé par: Magid, Enzo, Monis"
    ]

    if ChoixHéros=="1":#Affichage du texte pour Salie
        for phrase in trame_apres_boss:
            print(phrase) 
            time.sleep(2)
  
    elif ChoixHéros=="2":#Affichage du texte pour Magnus
        trame_apres_boss[1] = "Son pouvoir s'infiltre à travers les muscles de Magnus."                                                                     #modification texte à la ligne 1
        trame_apres_boss[2] = "Il court et bondit sur les décombres, brandit son épée et effectue un saut magistral d'une 40taine de mètres"                #modification texte à la ligne 5
        trame_apres_boss[5] = "La lumiere se dissipe. Derrière Magnus, le corps du fantôme, inerte, se désintègre"                                          #modification texte à la ligne 
        trame_apres_boss[6] = "Magnus ouvre le coffre sous le trone du roi, prend le joyau maudit et le détruit. Sauvant ainsi la contrée et ses habitants" #modification texte à la ligne 6
        for phrase in trame_apres_boss:
            print(phrase)   
            time.sleep(2)


#ZONE 1
trame_introduction()       #présentation de l'intrigue                                                           
commencement_histoire()    #ici-->rencontre de Garry (chasseurs de monstres)     
transition_1er_combat()    # ici-->entraînement terminé/premier combat                                           
mainCombat(1, "un Gobelin")#combat vs Gobelin LVL 1                                                              
Trésor(1)                  #coffre au trésor    
fin_introduction()         #ici-->le héros quitte le village.                                                                                   


#ZONE 2
prairie_niveau_1()           #--> prairie  niveau 1
mainCombat(2, "Stritominus") #Combat contre le Stritominus/ LVL 2
Trésor(2)         
prairie_niveau_2()               #dernier combat prairie/ LVL 2
mainCombat(2, "Stritominus_2.0") #Combat vs le dernier Stritominus
Trésor(2)       
print("C'est terminé... j'ai réussi...") #réplique apres 2e combat prairie
for i in range(5):#sauter 5 lignes
    print()        


#ZONE 3
foret_partie_1()            #nouvelle zone => foret / personnage passe une nuit dans la foret
foret_partie_2_combat_fin() #intrigue menant au combat vs mordeur  LVL 3
mainCombat(3, "le Mordeur") #Combat vs le Mordeur
réplique_apres_combat_mordeur()
Trésor(3)  

#ZONE 4
montagne()              #combat vs Orc et son loup
mainCombat(4, "l'Orc")  #Combat contre l'Orc LVL 4
Trésor(4)               #coffre en récompense               
montagne_fin()          #héros chevauche le loup de l'Orc jusqu'à la zone suivante


#ZONE 5
Château_abandonné()                           #héros arrive au château et le fouille pour trouver le joyau. Apparition du Chevalier Fantôme (BOSS FINAL)
mainCombat(5, "le Boss: le Chevalier Fantôme")#Combat vs Chevalier Fantôme LVL 5
Trésor(5)                                     #coffre en récompense
trame_finale_apres_boss()                     #trame de fin après le combat et fin de l'histoire

In [ ]:
#EXPLICATION POUR EXPOSE: LISTE / TIME.SLEEP() / TIME.SLEEP() INTERACTIF

def fin_introduction():#TIME.SLEEP() INTERACTIF

    répliques = [
    "Salie:",
    "OUAHH ! Je viens vraiment de tuer ce monstre !!?",
    "Oh ! Qu'est ce qui scintille juste là ?", 
    "*Lorsque vous neutralisez une créature, vous obtenez des récompenses.",
    "Cool, je pourrais en avoir besoin plus tard. ",
    "Garry:",
    " ",
    "Pas mal pour une première. Mais On dirait qu'on a eu chaud. On est pas en sécurité ici.",
    "L'avenir de notre village est incertain, il repose entre tes mains désormais. Va! Et ne reviens pas sans avoir détruit ce joyau !",
    "Salie:",
    "Et dire que je voulais juste une retraite paisible...",
    "Garry:",
    "Mais après tout, ce joyau n'apporte rien de bon. Les créatures s'excitent et se rapprochent du village.",
    "Salie:",
    "C'est compris Garry ! Dis à tout le monde qu'ils n'ont rien à craidre car je suis là ! "       
    " ",
    "*Ainsi le héros commence sa quête.",
    " ",
    ]

    if ChoixHéros=="1":#Affichage du texte pour Salie
        for phrase in répliques:
            print(phrase)
            time.sleep(2)
            input("Tapez ENTRER")

    elif ChoixHéros=="2":#Affichage du texte pour Magnus
        répliques[0] = "Magnus:"                                                                  #modification texte à la ligne 0
        répliques[1] = "OUAHH OK ! C'est bien plus impressionnant que ce que je pensais...°~°"    #modification texte à la ligne 1
        répliques[9] = "Magnus:"                                                                  #modification texte à la ligne 9
        répliques[13] = "Magnus:"                                                                 #modification texte à la ligne 13
        répliques[14] = "J'ai ressentis les fruits de mon entrainement aujoud'hui... Ca va mal finir pour ces fichus monstres !" #modification texte à la ligne 14
        for phrase in répliques:
            print(phrase)
            time.sleep(2)
            input("Tapez ENTRER")

def combat_introduction():#TIME.SLEEP() SIMPLE
    trame = [
    "*6 mois se sont écoulés",
    "Garry:",
    "Te voilà novice dans le rang des chasseurs. Chaque cible que tu élimineras te fera monter en rang. Ton savoir n'en sera qu'amélioré...",
    "Sache tout de même que ton premier but est de protéger le village.",
    "Comme par ton entraînement tu a su le comprendre, un grand pouvoir implique de grandes responsabilités...",
    "*Soudainement, un rugissement provenant du coeur du village attire votre attention.",
    "Oh non...",
    "Garry:", 
    "C'est l'occasion idéale, va sauver notre village et rapporte moi ce joyau!!",
    " ",
    ]
    for phrase in trame:   #prend les répliques présentes dans la liste
            print(phrase)  #les affiches une par une 
            time.sleep(2.5)#avec une intervalle de 2,5 secondes entre chaque 

combat_introduction()#TIME.SLEEP() SIMPLE
fin_introduction()   #TIME.SLEEP() INTERACTIF
       

In [ ]:
#TEST DE COMBAT 1/2
def combat_introduction():
    trame = [
    "*6 mois se sont écoulés",
    "Garry:",
    "Te voilà novice dans le rang des chasseurs. Chaque cible que tu élimineras te fera monter en rang. Ton savoir n'en sera qu'amélioré...",
    "Sache tout de même que ton premier but est de protéger le village.",
    "Comme par ton entraînement tu a su le comprendre, un grand pouvoir implique de grandes responsabilités...",
    "*Soudainement, un rugissement provenant du coeur du village attire votre attention.",
    "Oh non...",
    "Garry:", 
    "C'est l'occasion idéale, va sauver notre village et rapporte moi ce joyau!!",
    " ",
    ]
    for phrase in trame:   #prend les répliques présentes dans la liste
            print(phrase)  #les affiches une par une 
            time.sleep(2.5)#avec une intervalle de 2,5 secondes entre chaque 
#premier_combat doit suivre ici                    
def fin_introduction():

    répliques = [
    "Salie:",
    "OUAHH ! Je viens vraiment de tuer ce monstre !!?",
    "Oh ! Qu'est ce qui scintille juste là ?", 
    "*Lorsque vous neutralisez une créature, vous obtenez des récompenses.",
    "Cool, je pourrais en avoir besoin plus tard. ",
    "Garry:",
    " ",
    "Pas mal pour une première. Mais On dirait qu'on a eu chaud. On est pas en sécurité ici.",
    "L'avenir de notre village est incertain, il repose entre tes mains désormais. Va! Et ne reviens pas sans avoir détruit ce joyau !",
    "Salie:",
    "Et dire que je voulais juste une retraite paisible...",
    "Garry:",
    "Mais après tout, ce joyau n'apporte rien de bon. Les créatures s'excitent et se rapprochent du village.",
    "Salie:",
    "C'est compris Garry ! Dis à tout le monde qu'ils n'ont rien à craidre car je suis là ! "       
    " ",
    "*Ainsi le héros commence sa quête.",
    " ",
    ]

    if ChoixHéros=="1":#Affichage du texte pour Salie
        for phrase in répliques:
            print(phrase)
            time.sleep(2)
            input("Tapez ENTRER")

    elif ChoixHéros=="2":#Affichage du texte pour Magnus
        répliques[0] = "Magnus:"                                                                  #modification texte à la ligne 0
        répliques[1] = "OUAHH OK ! C'est bien plus impressionnant que ce que je pensais...°~°"    #modification texte à la ligne 1
        répliques[9] = "Magnus:"                                                                  #modification texte à la ligne 9
        répliques[13] = "Magnus:"                                                                 #modification texte à la ligne 13
        répliques[14] = "J'ai ressentis les fruits de mon entrainement aujoud'hui... Ca va mal finir pour ces fichus monstres !" #modification texte à la ligne 14
        for phrase in répliques:
            print(phrase)
            time.sleep(2)
            input("Tapez ENTRER")

combat_introduction() 
mainCombat(1, "Gobelin")
Trésor(1)                    
fin_introduction()

In [ ]:
#TEST DE COMBAT 2/2

def Château_abandonné():
    trame_château = [
    "Salie:",
    "Eh bas voilà vous pouvez bien servir à quelque chose des fo..",
    "*En voyant le château, le monstre déguerpit à toute vitesse",
    "Ok.. Ca n'annonce rien de bon.",
    "Je sais que je suis intimidante mais à ce point là ?",
    "*Les poils de Salie se hérissent",
    "*Une pression hors norme provient de l'intérieur du château",
    "Salie:",
    "Oh ! '°'",
    "Le château est immensément grand",
    "La poussière omniprésente est la marque du temps passé à l'abandon et à la mercie des créatures prédatrices environnantes",
    "Salie *craintive:",
    "Ca y est... nous y sommes.",
    " ",
    " ",
    ]

    if ChoixHéros=="1":#Affichage du texte pour Salie
            for phrase in trame_château:
                print(phrase)   #affiche texte
                time.sleep(1.75)#temps d'attente avant de dérouler la prochaine phrase
  
    if ChoixHéros=="2":#Affichage du texte pour Magnus
        trame_château[0] = "Magnus:"                                                             #modification texte à la ligne 0
        trame_château[3] = "Bas.. A bas d'accord. Je suis trop imposant pour toi c'est ça !?"    #modification texte à la ligne 3
        trame_château[4] = "C'est ça, cours flipp.."                                             #modification texte à la ligne 4                                         
        trame_château[5] = "*Les poils de Magnus se hérissent"                                   #modification texte à la ligne 5        
        trame_château[7] = "Magnus:"                                                             #modification texte à la ligne 8
        trame_château[11] = "Magnus *craintif:"                                                  #modification texte à la ligne 11
        for phrase in trame_château:
            print(phrase)   
            time.sleep(1.75)

    print("Chargement de la zone finale")
    chargement = [
    ".....",
    ".....",
    ".....",
    ]
    for element in chargement:    #les elements: "...", "...", "...", " ",
        for character in element: #chaque caractere (les points)
            print(character, end = "", flush = True)#pour faire afficher tous caracteres sur la même ligne
            time.sleep(0.60)#attente de 0,6 secondes 


    trame_entrée_château = [
    " ",
    "*Cinématique", 
    "Le héros se dirige vers l'entrée du château d'un pas craintif",
    "En arrivant devant la porte d'entrée:",
    "Salie:",  
    "Qu'est ce qui pourrait bien se caher derrière ?",
    "De toute façon il ne me semble pas qu'une bête se plairait dans ce château, endroit reculé..spacieux..et inhabité...",
    "Parfait pour y établir son nid...",
    "Une fois dedans, des dizaines d'ossements humains et animales jonchent le sol.",
    "Salie:",
    "Oh ! Ca empeste à la mort ici.",
    "Il y a un étage composé de plusieurs salles",
    "Peut être y trouverez-vous des indices sur la localisation du joyau...",
    ]

    if ChoixHéros=="1":#Affichage du texte pour Salie
            for phrase in trame_entrée_château:
                print(phrase) 
                time.sleep(2)
  
    if ChoixHéros=="2":#Affichage du texte pour Magnus
        trame_entrée_château[4] = "Magnus:" #modification texte à la ligne 3
        trame_entrée_château[9] = "Magnus" #modification texte à la ligne 3
        for phrase in trame_entrée_château:
            print(phrase)   
            time.sleep(1.75)

    trame_fouille = [
    "Vous allez au premier étage.",
    "en entrant dans une bibliothèque",
    "Vous trouvez plusieurs étagères remplies de livres et de parchemins. Au centre de la salle, un pupitre  et une lettre",
    "lettre*: Sur ordre du roi et en tant que son valet, j'ai mis en lieu sûr sa couronne",
    "Dans un coffre caché sous le trone même ! Qui aurait pû y penser ? :)",
    "la clé se trouve là où il n'y pas souvent grand monde, la...",
    "*le texte s'arrête là. Il a dû se passer quelque chose au moment où l'auteur écrivait",
    " ",
    "Vous entrez dans la cuisine",
    "Rien de plus déprimant qu'une cuisine sans rien à manger...",
    "Pas d'indices",
    " ",
    "Vous entrez dans la chambre d'ami.",
    "En vous allongeant sur le lit poussiéreux, quelque chose vous dérange sous le coussin presque entièrement déplumé de l'intérieur:",
    "En le soulevant, vous voyez une clé étrange.",
    "Vous avez obtenu: clé de coffre !",
    ]
    for phrase in trame_fouille:
        print(phrase)
        time.sleep(2)  
    

    trame_combat_dragon = [
    "*Soudain vous êtes surpris par un cri effrayant",
    "*Le château commence à s'écrouler",
    "Salie:",
    "AAAA ! Qu'est ce que c'est !?",
    "Un corps céleste jaillit du sol, explosant la toiture  du château !",
    "Salie:", 
    "Ca y est, c'est maintenant !",
    "BOSS FINAL: Chevalier fantôme",  
    "Salie:",
    "A nous deux sale monstre !!!",
    ] 
    if ChoixHéros=="1":#Affichage du texte pour Salie
        for phrase in trame_combat_dragon:
            print(phrase) 
            time.sleep(2)
  
    if ChoixHéros=="2":#Affichage du texte pour Magnus
        trame_combat_dragon[2] = "Magnus:" #modification texte à la ligne 3
        trame_combat_dragon[5] = "Magnus"  #modification texte à la ligne 5
        trame_combat_dragon[8] = "Magnus"  #modification texte à la ligne 8
        for phrase in trame_combat_dragon:
            print(phrase)   
            time.sleep(2)
#combat final##CODE COMBAT
def trame_finale_apres_boss():
    trame_apres_boss = [
    "L'épée scintille ! ",
    "Son pouvoir s'infiltre à travers les muscles de Salie.",
    "Elle court et bondit sur les décombres, brandit son épée et effectue un saut magistral d'une 40taine de mètres",
    "Le chevalier fantôme prépare son attaque ultime en volant jusqu'au ciel puis fonce en flèche en direction du château", 
    "Slaaash !!! Une lueur éclatante jaillit de cette impact dévastateur !",
    "La lumiere se dissipe. Derrière Salie, le corps du fantôme, inerte, se désintègre",
    "Salie ouvre le coffre sous le trone du roi, prend le joyau maudit et le détruit.²", 
    "*Les montres perdent leur pouvoir et deviennent innoffensifs ",
    "Sauvant ainsi la contrée et ses habitants",
    " ",
    "FIN"
    " ",
    "                       Réalisé par: Magid, Enzo, Monis"
    ]

    if ChoixHéros=="1":#Affichage du texte pour Salie
        for phrase in trame_apres_boss:
            print(phrase) 
            time.sleep(2)
  
    if ChoixHéros=="2":#Affichage du texte pour Magnus
        trame_apres_boss[1] = "Son pouvoir s'infiltre à travers les muscles de Magnus."                                                                     #modification texte à la ligne 1
        trame_apres_boss[2] = "Il court et bondit sur les décombres, brandit son épée et effectue un saut magistral d'une 40taine de mètres"                #modification texte à la ligne 5
        trame_apres_boss[5] = "La lumiere se dissipe. Derrière Magnus, le corps du fantôme, inerte, se désintègre"
        trame_apres_boss[6] = "Magnus ouvre le coffre sous le trone du roi, prend le joyau maudit et le détruit. Sauvant ainsi la contrée et ses habitants" #modification texte à la ligne 6
        for phrase in trame_apres_boss:
            print(phrase)   
            time.sleep(2)

Château_abandonné()                           #héros arrive au château et le fouille pour trouver le joyau. Apparition du Chevalier fantôme (BOSS FINAL)
mainCombat(5, "le Boss: le Chevalier Fantôme")#Combat vs Chevalier Fantôme
Trésor(5) 
trame_finale_apres_boss()                     #trame de fin après le combat et fin de l'histoire